In [111]:
from PyPDF2 import PdfReader,PdfWriter
import re
import pandas as pd
import os

In [124]:
os.path.exists("Files/")

True

In [136]:
skipped_files = {
        'file_path': []
}

In [138]:
for subdir,dirs,files in os.walk("Files/"):
    Folder = subdir.replace("Files/","")
    output_folder = "OutputFiles/" + Folder
    if files != []:
        for file in files:
            skipped=False
            file_path = subdir + "/" + file
            print(file_path)
            print(output_folder)
            if not os.path.exists(output_folder):
                os.mkdir(output_folder)
            #Read PDF
            reader = PdfReader(file_path)
            last_page = (len(reader.pages))
            page = reader.pages[0]
            text = page.extract_text()
            text = re.split('(?<=[\d])\s*((?=Event)|(?=Abstract))', text)
            text[0] = text[0].split("\n")[1]
            pdf_pages = {
                'name': [],
                'first_page': [] 
            }
            for i in range(0,len(text)):
                if text[i] != '':
                    value = re.split('\.\.+', text[i])
                    if len(value)<2:
                        skipped=True
                        break
                    name = value[0]
                    name = name.replace('\n','')
                    page = value[1]
                    page = page.replace('\n','')
                    pdf_pages['name'].append(name)
                    pdf_pages['first_page'].append(int(page))
            print(skipped,value)
            if skipped:
                skipped_files['file_path'].append(file_path)
                continue
            pd_pages = pd.DataFrame(pdf_pages)
            pd_pages['last_page'] = pd_pages['first_page'].shift(-1)
            pd_pages.iloc[len(pd_pages)-1,2] = last_page + 1
            pd_pages['last_page'] = pd_pages['last_page'] - 1

            for i in range(len(pd_pages)):
                writer = PdfWriter()
                page_range = range(pd_pages.iloc[i,1], int(pd_pages.iloc[i,2]) + 1)
                for page_num, page in enumerate(reader.pages, 1):
                    if page_num in page_range:
                        writer.add_page(page)
                if len(pd_pages.iloc[i,0])>100:
                    file_name_temp =  pd_pages.iloc[i,0]
                    file_name_temp = file_name_temp[0:70]
                    file_name_temp = re.sub("[^0-9a-zA-Z]","",file_name_temp)
                else:
                    file_name_temp =  pd_pages.iloc[i,0]
                    file_name_temp = re.sub("[^0-9a-zA-Z]","",file_name_temp)
                file_name = output_folder + "/" + file_name_temp
                with open(f'{file_name}.pdf', 'wb') as out:
                    writer.write(out)
            

Files/Agilent Technologies, Inc/Agilent Technologies, Inc_20030101_20060101.pdf
OutputFiles/Agilent Technologies, Inc
True ['EventBriefofCadenceNamesMichaelFisterCEOandRayBinghamChairmanoftheBoardConferenceCall-Final\n99']
Files/Agilent Technologies, Inc/Agilent Technologies, Inc_20060101_20090101.pdf
OutputFiles/Agilent Technologies, Inc
False ['EventBriefofQ12006AgilentTechnologiesInc.EarningsConferenceCallandAnalystMeeting-Final', '157']
Files/Agilent Technologies, Inc/Agilent Technologies, Inc_20090101_20120101.pdf
OutputFiles/Agilent Technologies, Inc
False ['EventBriefofQ12009AgilentTechnologiesInc.EarningsConferenceCall-Final', '156']
Files/Agilent Technologies, Inc/Agilent Technologies, Inc_20120101_20150101.pdf
OutputFiles/Agilent Technologies, Inc
True ['EventBriefofAgilentTechnologiestoSeparateintoTwoIndustry-LeadingPublicCompaniesConferenceCall-Final.\n88']
Files/Agilent Technologies, Inc/Agilent Technologies, Inc_20150101_20180101.pdf
OutputFiles/Agilent Technologies, Inc


In [116]:
skipped_files_pd = pd.DataFrame(skipped_files)

In [120]:
for i in range(len(skipped_files_pd)):
    print(skipped_files_pd.iloc[i,0])

Files/Blackstone Inc/Blackstone Inc_20090101_20120101.pdf
Files/Cadence Design Systems, Inc/Cadence Design Systems, Inc_20030101_20060101.pdf
Files/Comcast Corporation/Comcast Corporation_20060101_20090101.pdf
Files/Costco Wholesale Corporation/Costco Wholesale Corporation_20060101_20090101.pdf
Files/Costco Wholesale Corporation/Costco Wholesale Corporation_20120101_20150101.pdf
Files/HCA Healthcare, Inc/HCA Healthcare, Inc_20090101_20120101.pdf
Files/Intel Corporation/Intel Corporation_20020101_20030101.pdf
Files/Lam Research Corporation/Lam Research Corporation_20090101_20120101.pdf
Files/Microsoft Corporation/Microsoft Corporation_20120101_20150101.pdf
Files/Regeneron Pharmaceuticals, Inc/Regeneron Pharmaceuticals, Inc_20150101_20180101.pdf
Files/The Boeing Company/The Boeing Company_20090101_20120101.pdf


In [139]:
count = 0
for root_dir, cur_dir, files in os.walk(r'OutputFiles/'):
    count += len(files)
print('file count:', count)

file count: 3835
